In [19]:
import pandas as pd 
import numpy as np

from rake_nltk import Rake
rakers = Rake()

import re 
import string

import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import spacy
nlp = spacy.load('en_core_web_sm')

import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

cv = CountVectorizer(max_df=0.95, min_df=2, stop_words="english")

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec.load("models/cosine_doc2vec.model")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub('\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub('\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

# combo['cleaned'] = combo['excerpt'].apply(lambda x: clean_string(x, stem='Stem'))

In [18]:
# Load data 

# read = pd.read_csv('https://raw.githubusercontent.com/joshnicholas/article_reccomendation/main/archive/read.csv')
# unread = pd.read_csv('https://github.com/joshnicholas/article_reccomendation/blob/main/archive/unread.csv?raw=true')

vectored = pd.read_csv('archive/vectored.csv')

# print(vectored.columns)
# 'status', 'resolved_title', 'resolved_url', 'excerpt', 'cleaned',
#        'card2vec'

In [16]:

### Clean excerpts

# combo['cleaned'] = combo['cleaned'].astype(str)

# card2vec = [model.infer_vector(combo.iloc[i]['cleaned'].split(' '))
#             for i in range(0,len(combo))]


### Create new features to help with machine learning

vectored['title_count'] = vectored['resolved_title'].str.split(" ").str.len()

vectored = vectored[:1]


listo = []
for index, row in vectored.iterrows():

    texto = row['card2vec'].replace("[", '').replace("]", '').split(",")

    status =  row['status']
    titlo = row['resolved_title']
    title_count = row['title_count']
    urlo = row['resolved_url']

    excerpt = row['excerpt']

    a = rakers.extract_keywords_from_text(excerpt)
    keywords = rakers.get_ranked_phrases()

    data = [{'resolved_title': titlo,
    'resolved_url': urlo,
    'keywords':  }]

    inter = pd.DataFrame.from_records(data)

    listo.append(inter)




p = combo

# print(p)
# print(p.columns)
# print(p['title_count'])


['think donald trump', 'wing influencers', 'raising questions', 'political discussion', 'electoral success', '’', 'whether', 'translate', 'time', 'spend', 'right', 'november', 'november', 'need', 'listen', 'liberals', 'get', 'facebook', 'facebook', 'elected', 'dominating']
Right-wing influencers are dominating the political discussion on Facebook, raising questions about whether it will translate into electoral success in November. Listen, liberals. If you don’t think Donald Trump can get re-elected in November, you need to spend more time on Facebook.


In [ ]:
read = vectored.loc[vectored['status'] == 1].copy()
unread = vectored.loc[vectored['status'] == 0].copy()

### Sample the data to get 700 of each 

read = read.sample(n=7000)
unread = unread.sample(n=7000)

read = read[['status', 'resolved_title', 'excerpt', 'word_count']]
unread = unread[['status', 'resolved_title', 'excerpt', 'word_count']]

combo = unread.append(read)